## 線形計画問題
### 標準系の線形計画問題
- 変数が非負で，等式制約しかもたない問題
$$
\mbox{minimize} \; \; \boldsymbol{c}^{\mbox{T}}\boldsymbol{x} \\
\mbox{subject to} \; \; A \boldsymbol{x} = \boldsymbol{b} \\
\boldsymbol{x} \geq \boldsymbol{0}
$$
where $\boldsymbol{c} \in \mathbb{R}^n, \; \boldsymbol{b} \in \mathbb{R}^m, \; A \in \mathbb{R}^{m \times n} \, (m \leq n), \; \boldsymbol{x} \in \mathbb{R}^n$.
- PythonのパッケージPulpを使って問題を定義し，ソルバーに解かせる．

In [1]:
##### Packages #####
import numpy as np
import pulp as pp
import matplotlib.pyplot as plt
import time

In [2]:
##### Function #####
## Define Linear Problem
## key:
    # 1: Maximize
    # other: Minimize
## ineq:
    # 1: >=
    # -1: <= 
    # other: ==
## Output:
    # X_lst: variable, obj_val: objective value, time_solve: the time to solve
def Linear_problem(matrixA, vectorb, vectorc, key, ineq, problem_name):
    if key == 1:
        problem = pp.LpProblem(problem_name, pp.LpMaximize)
    else:
        problem = pp.LpProblem(problem_name, pp.LpMinimize)
    ## list of variables
    x = [pp.LpVariable("x(%s)"%i, cat="Continuous") for i in range(len(vectorc))]
    ## objective function
    objective = pp.lpSum(vectorc[i] * x[i] for i in range(len(vectorc)))
    problem += objective
    ## constraints
    ## it can be adapted if all constraints has the same inequality
    for i in range(len(matrixA)):
        if ineq == -1:
            cst = pp.lpSum(matrixA[i][j] * x[j] for j in range(len(x))) <= vectorb[i]
        elif ineq == 1:
            cst = pp.lpSum(matrixA[i][j] * x[j] for j in range(len(x))) >= vectorb[i]
        else:
            cst = pp.lpSum(matrixA[i][j] * x[j] for j in range(len(x))) == vectorb[i]
        problem += cst
    for i in range(len(x)):
        cst = x[i] >= 0
        problem += cst
    time_begin = time.time()
    ## execute problem
    status = problem.solve()
    time_end = time.time()
    time_solve = time_end - time_begin

    ## print result
    for i in range(len(x)):
        print("x[%d]: %f" % (i,x[i].value()))
    print("objective: %f" % objective.value())
    print("time: %f" % time_solve)

    ## Output
    # the list of x value
    X_lst = []
    for i in range(len(x)):
        X_lst.append(x[i].value())
    obj_val = objective.value()
    return X_lst, obj_val, time_solve

### 例題
次の生産計画問題を考える．
$$
\mbox{maximize} \; \; 4x_1 + 5x_2 \\
\mbox{subject to} \; \; 2.5x_1 + 5x_2 \leq 350 \\
5x_1 + 6x_2 \leq 450 \\
3x_1 + 2x_2 \leq 240 \\
x_1, x_2 \geq 0
$$
行列$A$, ベクトル$\boldsymbol{b}, \boldsymbol{c}, \boldsymbol{x}$を，
$$
A = \left(\begin{array}{cc}
2.5 & 5 \\
5 & 6 \\
3 & 2
\end{array}\right), \; \boldsymbol{b} = \left(
    \begin{array}{c}
    350 \\
    450 \\
    240
    \end{array}
    \right), \; \boldsymbol{c} = \left(
        \begin{array}{c}
        4 \\
        5
        \end{array}
        \right), \; \boldsymbol{x} = \left(
            \begin{array}{c}
            x_1 \\
            x_2
            \end{array}
            \right)
$$
とすると，この問題は，
$$
\mbox{maximize} \; \; \boldsymbol{c}^{\mbox{T}}\boldsymbol{x} \\
\mbox{subject to} \; \; A\boldsymbol{x} \leq \boldsymbol{b}\\
\boldsymbol{x} \geq \boldsymbol{0}
$$
となる．

In [3]:
# 標準形に直さずに定義
## parameter
A = np.array([[2.5, 5], [5, 6], [3, 2]])
b = np.array([[350], [450], [240]])
c = np.array([[4], [5]])
xlst, obj, t_solve = Linear_problem(A, b, c, 1, -1, "Product")

x[0]: 15.000000
x[1]: 62.500000
objective: 372.500000
time: 0.007357


上の問題を標準形に変形する．
$$
\mbox{minimize} \; \; -4x_1 - 5x_2 \\
\mbox{subject to} \; \; 2.5x_1 + 5x_2 + x_3 = 350 \\
5x_1 + 6x_2 + x_4 = 450 \\
3x_1 + 2x_2 + x_5 = 240 \\
x_1, x_2, x_3, x_4, x_5 \geq 0
$$

行列$A$, ベクトル$\boldsymbol{b}, \boldsymbol{c}, \boldsymbol{x}$を，
$$
A = \left(\begin{array}{ccccc}
2.5 & 5 & 1 & 0 & 0 \\
5 & 6 & 0 & 1 & 0 \\
3 & 2 & 0 & 0 & 1
\end{array}\right), \; \boldsymbol{b} = \left(
    \begin{array}{c}
    350 \\
    450 \\
    240
    \end{array}
    \right), \; \boldsymbol{c} = \left(
        \begin{array}{c}
        -4 \\
        -5 \\
        0 \\
        0 \\
        0
        \end{array}
        \right), \; \boldsymbol{x} = \left(
            \begin{array}{c}
            x_1 \\
            x_2 \\
            x_3 \\
            x_4 \\
            x_5
            \end{array}
            \right)
$$
とすると，この問題は，
$$
\mbox{minimize} \; \; \boldsymbol{c}^{\mbox{T}}\boldsymbol{x} \\
\mbox{subject to} \; \; A\boldsymbol{x} = \boldsymbol{b}\\
\boldsymbol{x} \geq \boldsymbol{0}
$$
となる．

In [4]:
# 標準形に直して定義
## parameter
nA = np.array([[2.5, 5, 1, 0, 0], [5, 6, 0, 1, 0], [3, 2, 0, 0, 1]])
nb = np.array([[350], [450], [240]])
nc = np.array([[-4], [-5], [0], [0], [0]])
nxlst, nobj, nt_solve = Linear_problem(nA, nb, nc, 0, 0, "Product")

x[0]: 15.000000
x[1]: 62.500000
x[2]: 0.000000
x[3]: 0.000000
x[4]: 70.000000
objective: -372.500000
time: 0.008561
